# Prepear test dataset
Format - {word}{several spaces}{answer-words separeted by comma}

In [9]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install pandas
import pandas as pd

In [11]:
test_incorrect_file = open('../data/test_sample_incorrect.txt', 'r', encoding="utf8")
test_incorrect_lines = test_incorrect_file.readlines()
splitted_test_incorrect_lines = list(map(lambda x: x.split(maxsplit=1), test_incorrect_lines))
test_incorrect_words = list(map(lambda x: x[0], splitted_test_incorrect_lines))
test_incorrect_answers = list(map(lambda x: list(map(lambda y: y.strip(), x[1].split(', '))), splitted_test_incorrect_lines))
test_df = pd.DataFrame(test_incorrect_words, columns=["test_word"])
test_df["answers"] = test_incorrect_answers


In [12]:
test_df.head()

,test_word,answers
0,эктренном,[экстренном]
1,синусовго,[синусового]
2,даным,"[данным, данный, данные]"
3,шнутография,[шунтография]
4,сникопальных,[синкопальных]


In [13]:
test_word_list = test_df["test_word"].tolist()

In [14]:
test_answers = test_df["answers"].tolist()

In [15]:
# Need to create test example for computing lexical precision
def compute_error_precision(original_word_list, corrected_word_list, answer_word_list):
    words_number = len(corrected_word_list)
    correct_words_number = 0
    print("original_word_list --- corrected_word --- answer_word_list")
    for i, corrected_word in enumerate(corrected_word_list):
        print(f"{original_word_list[i]} --- {corrected_word} --- {answer_word_list[i]}")
        if corrected_word in answer_word_list[i]:
            correct_words_number += 1
    print(f"Right corrected words count - {correct_words_number} of {words_number} total")
    return correct_words_number/words_number

In [16]:
!pip install tqdm
from tqdm import tqdm

In [17]:
def compute_all_metrics(spellchecker_function, original_word_list, answer_word_list):
    result = spellchecker_function()
    elapsed_time = result["elapsed"]
    corrected_word_list = result["corrected_word_list"]
    error_precision = compute_error_precision(original_word_list, corrected_word_list, answer_word_list)
    # TODO: Compute lexical precision
    lexical_precision = 0.0
    return elapsed_time / len(original_word_list), \
           error_precision, \
           lexical_precision, \
           original_word_list, \
           corrected_word_list,\
           answer_word_list

# Spellchecker prototype initialization

In [2]:
!pip install pymorphy2 
!pip install gensim
!pip install python-levenshtein
!pip install stringdist

  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp39-cp39-linux_x86_64.whl size=168591 sha256=1c310d11dcf61766d37a6f1c1c324b9daa53cd9a67f51440f95f1a00f5d9315f
  Stored in directory: /home/Dmitry.Pogrebnoy/.cache/pip/wheels/46/4a/6c/164a1d9dd67c82d208f19d869ad0a517a0c5a6117f608c53e6
Successfully built python-levenshtein


# Test SpellChecker Prototype v.2

In [3]:
from spellchecker_prototype_v2.spell_checker import SpellChecker

Download models (all files) from [link](https://drive.google.com/drive/folders/1ubVgiIC2pqDOpA-CbjEqV8Jo8uBbk0qi?usp=sharing) and move it to `data/spellchecker_prototype_v2/models` folder

In [4]:
def spellchecker_prototype_v2_correct_func():
    spellchecker_prototype = SpellChecker()
    input_text_for_prototype = " ".join(test_word_list)
    timer = tqdm()
    corrected_word_list = spellchecker_prototype.correct_words(input_text_for_prototype)
    return {"elapsed": timer.format_dict["elapsed"], "corrected_word_list": corrected_word_list}

spellchecker_prototype_v2_metrics = compute_all_metrics(spellchecker_prototype_v2_correct_func, test_word_list, test_answers)
spellchecker_prototype_v2_metrics

NameError: name 'compute_all_metrics' is not defined

# Test Symspellpy - https://github.com/mammothb/symspellpy

In [5]:
!pip install -U symspellpy

     |████████████████████████████████| 2.6 MB 508 kB/s            
     |████████████████████████████████| 57 kB 3.8 MB/s             
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp39-cp39-linux_x86_64.whl size=137366 sha256=1cdcde1ac87d9131a60044421ede35f55c5707edce581c0730bd56e85706181a
  Stored in directory: /home/Dmitry.Pogrebnoy/.cache/pip/wheels/72/16/b1/ea6472dedd6ac13ad789dd294871a76fd6906fca3c010347d1
Successfully built editdistpy


### Test Symspellpy lookup on basic dict

In [6]:
basic_frequency_dict = '../data/symspell/ru-100k.txt'

In [18]:
from symspellpy import SymSpell, Verbosity

def test_symspell_py_lookup(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)
        result.append(suggestions[0].term)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

def test_symspell_py_lookup_compound(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup_compound(word, max_edit_distance=2)
        result.append(suggestions[0].term)
    return {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}



In [19]:
test_symspell_py_lookup_result = compute_all_metrics(
    lambda : test_symspell_py_lookup(basic_frequency_dict, test_word_list), test_word_list, test_answers)
test_symspell_py_lookup_result

100%|██████████| 200/200 [00:00<00:00, 11850.49it/s]

original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синусовго --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- шнутография --- ['шунтография']
сникопальных --- сникопальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- подавай --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сентированный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стентированеим --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- догосптальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загрудиные --- ['загрудинные', 'загрудинный']
перферия --- периферия --- ['перифери

(8.65626335144043e-05,
 0.42,
 0.0,
 ['эктренном',
  'синусовго',
  'даным',
  'шнутография',
  'сникопальных',
  'посление',
  'ходавый',
  'анмнезе',
  'сентированный',
  'самоч',
  'перевдена',
  'стентированеим',
  'кардиол',
  'переведна',
  'догосптальном',
  'миним',
  'субфебриллитет',
  'загрудиные',
  'перферия',
  'течени',
  'оклюзия',
  'жнщина',
  'пциент',
  'зарегистирована',
  'сегмета',
  'клекте',
  'эпид',
  'интелектуально',
  'нижн',
  'прводилась',
  'неуст',
  'госптализирован',
  'вперые',
  'послеоперационом',
  'госпитализирова',
  'ухудщение',
  'рентг',
  'дальнешем',
  'троботическая',
  'многосоудистое',
  'миокрада',
  'сопрвождались',
  'дискмфорт',
  'потлтвостью',
  'левогожелудочка',
  'рание',
  'виед',
  'ангинозныеболи',
  'догоситальном',
  'ввполнен',
  'чаосв',
  'октазалась',
  'линика',
  'ммента',
  'проведеной',
  'выполненяи',
  'леченяи',
  'обтилась',
  'повтороно',
  'зарегистриованы',
  'отицает',
  'дальнейщем',
  'болисохраняются',
 

In [20]:
test_symspell_py_lookup_compound_result = compute_all_metrics(
    lambda : test_symspell_py_lookup_compound(basic_frequency_dict, test_word_list), test_word_list, test_answers)
test_symspell_py_lookup_compound_result

100%|██████████| 200/200 [00:00<00:00, 677.85it/s]

original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синус его --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- и фотография --- ['шунтография']
сникопальных --- они локальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- ход вый --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- центров данный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стен титрованием --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- до остальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- себе бриллиант --- ['субфебрилитет']
загрудиные --- за рудные --- ['загрудинные', 'загрудинный']
перферия --- периферия --- ['периф

(0.0014775395393371582,
 0.46,
 0.0,
 ['эктренном',
  'синусовго',
  'даным',
  'шнутография',
  'сникопальных',
  'посление',
  'ходавый',
  'анмнезе',
  'сентированный',
  'самоч',
  'перевдена',
  'стентированеим',
  'кардиол',
  'переведна',
  'догосптальном',
  'миним',
  'субфебриллитет',
  'загрудиные',
  'перферия',
  'течени',
  'оклюзия',
  'жнщина',
  'пциент',
  'зарегистирована',
  'сегмета',
  'клекте',
  'эпид',
  'интелектуально',
  'нижн',
  'прводилась',
  'неуст',
  'госптализирован',
  'вперые',
  'послеоперационом',
  'госпитализирова',
  'ухудщение',
  'рентг',
  'дальнешем',
  'троботическая',
  'многосоудистое',
  'миокрада',
  'сопрвождались',
  'дискмфорт',
  'потлтвостью',
  'левогожелудочка',
  'рание',
  'виед',
  'ангинозныеболи',
  'догоситальном',
  'ввполнен',
  'чаосв',
  'октазалась',
  'линика',
  'ммента',
  'проведеной',
  'выполненяи',
  'леченяи',
  'обтилась',
  'повтороно',
  'зарегистриованы',
  'отицает',
  'дальнейщем',
  'болисохраняются',


# Test SymSpellCppPy - https://github.com/viig99/SymSpellCppPy

In [22]:
!pip install wheel setuptools

In [29]:
!pip install --upgrade SymSpellCppPy

  Using cached SymSpellCppPy-0.0.14.tar.gz (5.4 MB)
  Preparing metadata (setup.py) ... done
  ERROR: Command errored out with exit status 1:
   command: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-h03nugqd/symspellcpppy_7de71a60b9a84cd9aba25d717b40bd4e/setup.py'"'"'; __file__='"'"'/tmp/pip-install-h03nugqd/symspellcpppy_7de71a60b9a84cd9aba25d717b40bd4e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-6n1izgqg
       cwd: /tmp/pip-install-h03nugqd/symspellcpppy_7de71a60b9a84cd9aba25d717b40bd4e/
  Complete output (45 lines):
  running bdist_wheel
  running build
  running build_ext
  Traceback (most recent call last):
    File "/tmp/pi

There was a problem with installing the package. It is still unclear what the reason is. However, this spellchecker should not provide more accuracy, only higher speed since it is implemented in C++. However, the speed of the Python implementation is still enough at this stage.  So I decided to leave it here for now and come back if there is a need for more speed.

# Test JunSpell - https://github.com/bakwc/JamSpell

#### Attention No Windows 10 support!

In [28]:
!pip install -I --no-cache-dir jamspell

     |████████████████████████████████| 174 kB 464 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp39-cp39-linux_x86_64.whl size=1901024 sha256=12f999b71c361ba070861480a02e92d3afc4e0f332bea2ec4b1851025bbabb54
  Stored in directory: /tmp/pip-ephem-wheel-cache-t_m_awed/wheels/fa/7e/8f/c23ae02f1556243d33cc2e3bddf98f8cc146e174e5855788b6
Successfully built jamspell
